In [2]:
import pandas as pd
import re

In [9]:
def is_valid_username(username):
    # Regular expression pattern for the username format BR123456-FirstLast.
    pattern = r"^[A-Z]{2}\d{6}-[A-Z][a-z]+([A-Z][a-z]+)$"
    # Use the re.match function to check if the username matches the pattern.
    if re.match(pattern, username):
        return True
    else:
        return False

In [25]:
def separate_parts(username):
    # Define a regular expression pattern to extract enrollment and name parts.
    pattern = r"^([A-Z]{2}\d{6})-([A-Z][a-z]+(?:[A-Z][a-z]+)*)$"
    
    # Use re.match to find the parts.
    match = re.match(pattern, username)
    
    if match:
        enrollment_part = match.group(1)
        name_part = match.group(2)
        # Split the name part into first and last names.
        name_parts = re.findall(r'[A-Z][a-z]*', name_part)
        
        # If there are two or more name parts, assume the last one is the last name.
        if len(name_parts) >= 2:
            first_name = name_parts[0]
            last_name = name_parts[-1]
        else:
            first_name = name_part  # If there's only one name part, consider it the first name.
            last_name = ""
        
        return enrollment_part, first_name, last_name
    else:
        return None, None, None

# Example usage:
username = "AB123456-JohnDoe"
enrollment, first_name, last_name = separate_parts(username)

if enrollment:
    enrollment="0818"+enrollment
    print(f"Enrollment: {enrollment}")
    print(f"First Name: {first_name}")
    print(f"Last Name: {last_name}")
else:
    print("Invalid username format.")

Enrollment: 0818AB123456
First Name: John
Last Name: Doe


In [24]:
# Define the file paths for your CSV files
roster_file_path = "Test-IIST.csv"
#file2_path = "file2.csv"

# Load the CSV files into pandas DataFrames
df1 = pd.read_csv(roster_file_path)
#df2 = pd.read_csv(file2_path)

# Specify the columns you want to match
#column_to_match1 = "A"
#column_to_match2 = "B"

# Merge the DataFrames based on the specified columns
#merged_df = pd.merge(df1, df2, left_on=column_to_match1, right_on=column_to_match2)

# Display the resulting merged DataFrame
#print(merged_df)
df1


,identifier,github_username,github_id,name
0,Abhishek Rajpoot,CS211009-AbhishekRajput,144265821,Deleted user
1,Anshul Sharma,0818cs211032-AnshulSharma,139057485,0818cs211032-AnshulSharma
2,Harsh Hatej,CS211066-HarshHatej,144616839,NaN
3,SIDDHESH JOSHI,abhishek9r,112405138,Abhishek Rajput


In [26]:
""" 
Verify identifiers with official names and point out unkown names
>Instructor should wait for roaster correction before assigning
Verify naming convention then extract 3 parts(ennrollment, FNAME, LNAME), point out those who have voilated convention
Verify enrollement if present in official records and point out those with wrong enrollement
also crosscheck names point out if wrong

"""

' \nVerify identifiers with official names and point out unkown names\n>Instructor should wait for roaster correction before assigning\nVerify naming convention then extract 3 parts(ennrollment, FNAME, LNAME), point out those who have voilated convention\nVerify enrollement if present in official records and point out those with wrong enrollement\nalso crosscheck names point out if wrong\n\n'